In [7]:
import os
from glob import glob

import dask
import dask.bag as db
import dask.dataframe as dd
from distributed import Client
from dask_jobqueue import SLURMCluster

from IPython.display import display

import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:
# Set LOCAL to True for single-machine execution while developing
# Set LOCAL to False for cluster execution
LOCAL = True
# LOCAL = False

if LOCAL:
    # This line creates a single-machine dask client
    client = Client()
else:    
    # This line creates a SLURM cluster dask and dask client
    # Logging outputs will be stored in /scratch/{your-netid}
    
    cluster = SLURMCluster(memory='64GB', cores=4, python='/scratch/work/public/dask/bin/python', 
                               local_directory='/tmp/{}/'.format(os.environ['SLURM_JOB_USER']),
                               job_extra=['--output=/scratch/{}/slurm-%j.out'.format(os.environ['SLURM_JOB_USER'])])

    cluster.submit_command = 'slurm'
    cluster.scale(100)

    display(cluster)
    client = Client(cluster)

display(client)

/ext3/miniconda3/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45801 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:34985 Dashboard: http://127.0.0.1:45801/status,Cluster Workers: 1 Cores: 1 Memory: 4.29 GB


In [ ]:
# ratings = dd.read_csv("/scratch/work/courses/DSGA1004-2021/movielens/ml-latest-small/ratings.csv")
ratings = dd.read_csv("/scratch/work/courses/DSGA1004-2021/movielens/ml-latest/ratings.csv")
# mov = dd.read_csv("/scratch/work/courses/DSGA1004-2021/movielens/ml-latest-small/movies.csv")
ratings = pd.DataFrame(ratings.compute())
# mov = pd.DataFrame(mov.compute())

In [ ]:
# mov
ratings.duplicated(subset=col).sum()

In [5]:
# ratings = ratings.merge(mov, how = 'right', on = "movieId")

In [6]:
ratings

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264
...,...,...,...,...
1988537,283228,8542,4.5,1379882795
1988538,283228,8712,4.5,1379882751
1988539,283228,34405,4.5,1379882889
1988540,283228,44761,4.5,1354159524


In [7]:

user = pd.DataFrame(ratings.userId.unique())
train_user, testval_user = train_test_split(user, test_size=0.4)


movie = pd.DataFrame(ratings.movieId.unique())
temp1_movie, temp2_movie = train_test_split(movie, test_size = 0.7)
train_set = ratings[ratings['userId'].isin(list(train_user[0]))]
testval_set = ratings[ratings['userId'].isin(list(testval_user[0]))]
train_sending = testval_set[testval_set['movieId'].isin(list(temp1_movie[0]))]
testval_cluster =testval_set[testval_set['movieId'].isin(list(temp2_movie[0]))]

temp = pd.DataFrame(testval_set.userId.unique())
val_user, test_user = train_test_split(temp, test_size = 0.5)
val_set = testval_cluster[testval_cluster['userId'].isin(list(val_user[0]))]
test_set = testval_cluster[testval_cluster['userId'].isin(list(test_user[0]))]

In [8]:
val_set

,userId,movieId,rating,timestamp
778,5,47,4.0,1209126049
780,5,293,3.5,1209040005
782,5,318,5.0,1209040104
783,5,527,4.5,1209126022
785,5,778,5.0,1209040034
...,...,...,...,...
1987969,283221,4963,3.0,1168346430
1987971,283221,5995,4.5,1168346995
1987972,283221,6539,3.5,1168346437
1987973,283221,30749,4.5,1168346921


In [9]:
test_set

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264
...,...,...,...,...
1988537,283228,8542,4.5,1379882795
1988538,283228,8712,4.5,1379882751
1988539,283228,34405,4.5,1379882889
1988540,283228,44761,4.5,1354159524


In [10]:
pd.DataFrame(train_set)

,userId,movieId,rating,timestamp
31,3,640,3.0,945141610
32,3,828,4.0,945141610
33,3,960,3.0,945141407
34,3,1221,4.0,945141407
35,3,1321,4.0,945141407
...,...,...,...,...
1988424,283227,3006,5.0,1346633534
1988425,283227,3256,5.0,1346633536
1988426,283227,4002,4.5,1346633623
1988427,283227,4321,4.5,1346633626


In [12]:
testval_set

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264
...,...,...,...,...
1988537,283228,8542,4.5,1379882795
1988538,283228,8712,4.5,1379882751
1988539,283228,34405,4.5,1379882889
1988540,283228,44761,4.5,1354159524


In [2]:
# concatenate back to train set
final_train = pd.concat([train_set, train_sending])

NameError: name 'pd' is not defined

In [1]:
final_train

NameError: name 'final_train' is not defined

In [171]:
# train_small_data = final_train
# val_small_data = val_set
# test_small_data = train_set
train_large_data = final_train
val_large_data = val_set
test_large_data = train_set

In [125]:
# train_small_data.to_csv("train_small_data")
# val_small_data.to_csv("val_small_data")
# test_small_data.to_csv("test_small_data")
train_large_data.to_csv("train_large_data")
val_large_data.to_csv("val_large_data")
test_large_data.to_csv("test_large_data")